# Chinese Text Classification

In [48]:
# scripting language : Python 3.6
# modules : pandas, numpy, sklearn, bs4, jieba 

import time
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

import re
from bs4 import BeautifulSoup
import jieba
import jieba.analyse

### Load data

First, we load data into dataframe X_train, y_train X_text, and label encode the classes into [0,1,2].

In [32]:
# load data to dataframe
df_train = pd.read_csv("data/offsite-tagging-training-set.csv",  header=0)
df_test = pd.read_csv("data/offsite-tagging-test-set.csv",  header=0)
print("Load {} training examples and {} test cases".format(len(df_train), len(df_test)))

y_train = df_train['tags']
X_train = df_train['text']
X_test = df_test['text']

# label encode the classes
le = LabelEncoder()
y_train = le.fit_transform(y_train)

print("\nShow first training example: \n", X_train[0], "\n\nClass label:", y_train[0])
print("\nClass labels [0,1,2] refer to ",le.inverse_transform([0, 1, 2]))

Load 3894 training examples and 974 test cases

Show first training example: 
 利物浦重賽擊敗乙組仔　英足盃過關 英格蘭足總盃第三圈今晨重賽，貴為英超勁旅的利物浦上場被乙組仔埃克斯特尷尬逼和，多獲一次機會的紅軍不敢再有差池。先有近期回勇的「威爾斯沙維」祖阿倫10分鐘開紀錄，加上兩個小將舒爾奧祖，及祖奧迪西拿下半場各入一球，以3比0擊敗對手，總算在主場挽 <p style="text-align: justify;">英格蘭足總盃第三圈今晨重賽，貴為英超勁旅的利物浦上場被乙組仔埃克斯特尷尬逼和，多獲一次機會的紅軍不敢再有差池。先有近期回勇的「威爾斯沙維」祖阿倫10分鐘開紀錄，加上兩個小將舒爾奧祖，及祖奧迪西拿下半場各入一球，以3比0擊敗對手，總算在主場挽回面子，下一圈對手為韋斯咸。</p> <p style="text-align: justify;">另一場英超球隊對壘，今季異軍突起的李斯特城戰至第三圈就宣告畢業。熱刺憑韓國前鋒孫興<U+615C>上半場遠射破網先開紀錄，換邊後此子助攻予中場查迪尼建功，令球隊以兩球輕取李斯特城，第四圈將面對英甲的高車士打。</p> 

Class label: 2

Class labels [0,1,2] refer to  ['梁振英' '美國大選' '足球']


### Text Preprocessing

Next, we write two methods to build our own stopword dictionary and perform the following text cleaning procedures.
* HTML tag removal using BeautifulSoup
* Remove all english letters, numbers, symbols using regular expression
* Perform chinese word segmentation using Jeiba
* Remove stopwords using customised chinese stopwords and symbols
* Join words with white space (for input to TfidfVectorizer in the next stage) 

In [39]:
def build_stopwords(filepath):
    '''
    Load customized stopwords.txt
    Return a dictionary of stopwords
    '''
    with open(filepath,'r') as file:
        return set([line.strip() for line in file])

In [40]:
def clean_text(text_array):
    '''
    Remove html tags, symbols, letters a-z A-Z, numbers
    Perform chinese word segmentation
    Remove stopwords 
    Return list of cleaned text
    '''
    start_time = time.time() 
    print("--- Text Preprocessing ---")
    
    cleaned_text_array = []
    
    for i in range(len(text_array)):
        raw_text = text_array[i]
        cleaned_text = BeautifulSoup(raw_text, 'lxml').get_text()  
        chinese_only = re.sub("[0-9a-zA-Z\xa0\r\n\t\u3000\u2000-\u206F\u2E00-\u2E7F\!#$%&()*+,\-.\/:;<=>?@\[\]^_`{|}~]", "", cleaned_text)
        words = list(jieba.cut(chinese_only, cut_all=False))
        words = [word for word in words if str(word) not in stopwords]
        cleaned_text_array.append(" ".join(words))
        
    print("--- %s seconds --- \n" % (time.time() - start_time))
    
    return cleaned_text_array

In [41]:
# build dictionary of stopwords from customized stopwords file
stopwords = build_stopwords(filepath='stopwords.txt')

# text preprocessing
X_train_cleaned = clean_text(X_train)
X_test_cleaned = clean_text(X_test)

print("\nShow first training example in raw text: \n", X_train[0])
print("\nShow first training example after text preprocessing: \n", X_train_cleaned[0])

--- Text Preprocessing ---
--- 52.093252182006836 seconds --- 

--- Text Preprocessing ---
--- 13.8717360496521 seconds --- 


Show first training example in raw text: 
 利物浦重賽擊敗乙組仔　英足盃過關 英格蘭足總盃第三圈今晨重賽，貴為英超勁旅的利物浦上場被乙組仔埃克斯特尷尬逼和，多獲一次機會的紅軍不敢再有差池。先有近期回勇的「威爾斯沙維」祖阿倫10分鐘開紀錄，加上兩個小將舒爾奧祖，及祖奧迪西拿下半場各入一球，以3比0擊敗對手，總算在主場挽 <p style="text-align: justify;">英格蘭足總盃第三圈今晨重賽，貴為英超勁旅的利物浦上場被乙組仔埃克斯特尷尬逼和，多獲一次機會的紅軍不敢再有差池。先有近期回勇的「威爾斯沙維」祖阿倫10分鐘開紀錄，加上兩個小將舒爾奧祖，及祖奧迪西拿下半場各入一球，以3比0擊敗對手，總算在主場挽回面子，下一圈對手為韋斯咸。</p> <p style="text-align: justify;">另一場英超球隊對壘，今季異軍突起的李斯特城戰至第三圈就宣告畢業。熱刺憑韓國前鋒孫興<U+615C>上半場遠射破網先開紀錄，換邊後此子助攻予中場查迪尼建功，令球隊以兩球輕取李斯特城，第四圈將面對英甲的高車士打。</p>

Show first training example after text preprocessing: 
 利物浦 重賽 擊敗 乙組 仔英足 盃 過關   英格蘭足 總 盃 第三圈 今晨 重賽 貴為 英超 勁 旅 利物浦 上場 乙 組仔 埃克斯 特 尷尬 逼 多獲 一次 機會的紅 軍 不敢 再有 差池 先有 近期 回勇 威爾斯沙維 祖阿倫 分鐘 開紀錄 加上 兩個 小將 舒爾奧祖 及祖奧 迪西 拿下 半場 各入 一球 以比擊敗 對 手 總算 主場 挽   英格蘭足 總 盃 第三圈 今晨 重賽 貴為 英超 勁 旅 利物浦 上場 乙 組仔 埃克斯 特 尷尬 逼 多獲 一次 機會的紅 軍 不敢 再有 差池 先有 近期 回勇 威爾斯沙維 祖阿倫 分鐘 開紀錄 加上 兩個 小將 舒爾奧祖 及祖奧 迪西 拿下 半場 各入 一球 以比擊敗 對 手 總算 主場

### Create Bag of Words
We are now ready to create bag of words. We use TfidfVectorizer from sklearn to convert list of cleaned text into numeric representation. Basically, it counts word occurance frequency and normalizes it by putting more weight on rare and meaningful words. After tf-idf transformation, we have created a dictionary of size 1425. Each text is now represented as a feature vector of 1425 dimensions.

Note that X_train_tfidf.shape = (3894, 1425). We use this for input of Randon Forest Classifer in the next stage.

In [42]:
# create Bag of Words using tf-idf transform
print("--- Create Bag of Words using TfidfVectorizer ---")

start_time = time.time() 
tfidf = TfidfVectorizer(tokenizer=lambda x: x.split(), lowercase=False, min_df=100) #minumum document frequency set to 100
X_train_tfidf = tfidf.fit_transform(X_train_cleaned)

print("--- %s seconds --- \n" % (time.time() - start_time))

print("Number of words in dictionary : {}".format(len(tfidf.get_feature_names())))
print("Show some words in dictionary : \n", tfidf.get_feature_names()[::100])

--- Create Bag of Words using TfidfVectorizer ---
--- 1.5314240455627441 seconds --- 

Number of words in dictionary : 1421
Show some words in dictionary : 
 ['一仗', '事', '傳球', '半場', '回合', '完成', '怎樣', '撕裂', '有些', '港元', '相比', '美國', '變得', '選戰', '風波']


### Random Forest Classification
Next, we fit the training set (X_train_tfidf, y_train) to random forest classifier. We search the best parameter set by using grid search on 10-fold cross-validation. The model is evaluated using accurany score.

In [82]:
# grid search for best paramter set of random forest classifier
print("--- Training Random Forest Classifier ---")

rf = RandomForestClassifier()
params = {"max_depth":[None, 10, 50],  #max depth of trees
          "n_estimators":[10, 100]  #no. of tress to ensemble
         } 

model = GridSearchCV(estimator=rf, param_grid=params, scoring='accuracy', cv=10, n_jobs=-1, verbose=0)
model.fit(X_train_tfidf, y_train)

print("\nThe best paramenter set is : \n", model.best_params_)
print("\nScores:")
for scores in model.grid_scores_:
    print(scores)

--- Training Random Forest Classifier ---

The best paramenter set is : 
 {'max_depth': None, 'n_estimators': 100}

Scores:
mean: 0.99024, std: 0.00396, params: {'max_depth': None, 'n_estimators': 10}
mean: 0.99230, std: 0.00381, params: {'max_depth': None, 'n_estimators': 100}
mean: 0.98151, std: 0.00636, params: {'max_depth': 10, 'n_estimators': 10}
mean: 0.99024, std: 0.00561, params: {'max_depth': 10, 'n_estimators': 100}
mean: 0.98819, std: 0.00652, params: {'max_depth': 50, 'n_estimators': 10}
mean: 0.99153, std: 0.00564, params: {'max_depth': 50, 'n_estimators': 100}


/Users/sukilau/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


### Prediction on Test Set

Finally, we make prediction on the test set using the best parameter model obtained by grid search. 

In [84]:
# make prediction on test set
print("--- Making prediction on test set ---")

prediction = model.predict(tfidf.transform(X_test_cleaned))
prediction_le = le.inverse_transform(prediction)
solution = pd.DataFrame(prediction_le, df_test['id'], columns = ["tag"])
solution.to_csv("prediction.csv")

print("Prediction on all test cases saved in prediction.csv\n")

print("Show prediction on all test cases:")
for i in range(len(X_test)):
    print(prediction_le[i],":", X_test[i][0:20], "...")

--- Making prediction on test set ---
Prediction on all test cases saved in prediction.csv

Show prediction on all test cases:
足球 : 南華添鋒力　簽前厄瓜多爾國腳保耶 港超勁 ...
梁振英 : 如果大學$0捐款　科大嶺南將蝕過千萬元  ...
足球 : 英超最強火力對碰　雙城爭冠靠鋒霸 英超今 ...
足球 : 【01球評】膺半程冠軍　阿仙奴今季不奪標 ...
梁振英 : 【書商失蹤】梁振英：希望失蹤的李波本人提 ...
梁振英 : 【施政盤點】三份施政報告　僅一半政策達標 ...
梁振英 : 【施政盤點】「治港絕招」　設19委員會　 ...
足球 : 高普首簽　「新馬迪」來季投紅軍 利物浦傷 ...
足球 : 「最潮主帥」鬥利物浦：我已領先1：0 英 ...
足球 : 紅軍超殘陣逼和英乙隊　高普：負擔不起重賽 ...
足球 : 【施丹上馬】皇馬六條A　退下來各自精彩　 ...
足球 : 【施丹上馬】踢而優則教　碧根鮑華告魯夫完 ...
足球 : 【01球評】施丹首戰　回歸原點抄足肥安　 ...
足球 : 新兵白鶴對辦　東方大破南華踞榜首現霸氣  ...
足球 : 【意甲半程總結】四軍混戰　拿玻里勢破祖記 ...
足球 : 【01球評】協同效應+品牌角力　美斯C朗 ...
足球 : 金球獎合併5年　首屆美斯得獎爭議最大 自 ...
梁振英 : 梁振英批司法覆核遭濫用　對政府代價大 現 ...
梁振英 : 【港大民調】支持度僅37%創新低　數據顯 ...
梁振英 : 【獨家民調】梁治三年　市民最不滿政制司法 ...
美國大選 : 奧巴馬炮轟特朗普耍選戰手段　承認未能團結 ...
梁振英 : 【施政報告】預留20億元成立創科創投基金 ...
梁振英 : 由象徵式政策到象徵式施政報告 師父教落， ...
梁振英 : 【施政報告】工聯會考慮不支持梁振英　梁連 ...
梁振英 : 港美同日發表施政藍圖　梁振英奧巴馬4大看 ...
梁振英 : 【獨家民調】青年人拒撐梁　無學生支持梁振 ...
足球 : 港中戰球迷噓國歌　FIFA再罰香港足總7 ...
梁振英 : 【01拆局】曾俊華民望雖高　未獲左派認可 ...
足球 : 【港足日與夜】從日與夜開始